Sunday 10/21:

- Set deadlines for when certain things should be done by



Monday 10/22:

- Created the repository and the ReadMe



Sunday 10/28:

- Tried to figure out what we should do about dangling links when making the matrices (which occur when people produce a target that was never tested as a cue) 
- Decided on two options: 
1. a matrix that just ignores dangling links (and in the case of a stochstic matrix normalizes the remaining values to sum to 1)
2. a matrix with entries for the dangling links. We talked about just having no out-edges from nodes corresponding to the dangling links. But now I'm thinking we could just give them links to all other nodes (with weights uniformly distributed in the case of a stochastic matrix)

- I wrote the functions to create the matrices for the first option

- We also discussed what form to load in the data in

Monday 10/29:

- Wrote methods to create the matrices (boolean and stochastic) for the second option from above. So the matrix has rows/cols for the dangling links, but right now these nodes have no out-going edges. So we still have to decide what we'll do for that and code it up